In [14]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import warnings
warnings.filterwarnings('ignore')




In [15]:
def parse_latlon(latlon_str):
    try:
        coords = []
        points = latlon_str.strip().split(":")
        for pt in points:
            if pt.strip() == "":
                continue  # Skip empty
            latlon = pt.strip().split(",")
            if len(latlon) != 2:
                continue  # Skip invalid pairs
            lat, lon = float(latlon[0]), float(latlon[1])
            coords.append((lon, lat))  # GeoJSON = (lon, lat)
        if len(coords) >= 3:
            return Polygon(coords)
        else:
            return None  # Invalid polygon
    except Exception as e:
        print("Error parsing:", latlon_str)
        print("Exception:", e)
        return None


In [16]:


# Load Excel and drop duplicate CCC+GEOFENCE rows
path = r'C:\Users\KRISHNENDU\Downloads\New folder\GEOFENCE.xlsx'
df = pd.read_excel(path)
df = df.drop_duplicates(subset=["CCC_NAME", "GEOFENCE_NAME"], keep="last")

# Apply the safe parser
df['geometry'] = df['LATLON'].apply(parse_latlon)

# Drop rows with invalid geometry
df = df[df['geometry'].notnull()]




In [17]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.set_crs(epsg=4326, inplace=True)
gdf.to_file("geofence_output.geojson", driver="GeoJSON")

